**Bibliografía:**
Practical Convolutional Neural Networks: Implement advanced deep learning models using Python

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [11]:
import tensorflow_datasets as tfds

ModuleNotFoundError: No module named 'tensorflow_datasets'

**Introduction to the MNIST dataset**

Aquí usamos MNIST (Instituto Nacional Modificado de Estándares y Technology), que consta de imágenes de números escritos a mano y sus etiquetas. Desde su lanzamiento en 1999, este conjunto de datos clásico se utiliza para comparar algoritmos de clasificación.

Los archivos de datos train.csv y test.csv constan de dígitos dibujados a mano, del 0 al 9 en forma de imágenes en escala de grises. Una imagen digital es una función matemática de la forma f (x, y) = valor de píxel. Las imágenes son bidimensionales.

Podemos realizar cualquier función matemática sobre la imagen. Al calcular el gradiente en la imagen, podemos medir qué tan rápido cambian los valores de los píxeles y la dirección en la que cambian. Para el reconocimiento de imágenes, convertimos la imagen a escala de grises para simplificar y tenemos un canal de color. La representación RGB de una imagen consta de tres canales de color: ROJO, AZUL y VERDE. En el esquema de color RGB, una imagen es una pila de tres imágenes ROJA, AZUL y VERDE. En un esquema de color en escala de grises, el color no es importante. Las imágenes en color son computacionalmente más difíciles de analizar porque ocupan más espacio en la memoria. La intensidad, que es una medida de la claridad y oscuridad de una imagen, es muy útil para reconocer objetos. En algunas aplicaciones, por ejemplo, al detectar líneas de carril en una aplicación de automóvil autónomo, el color es importante porque tiene que distinguir los carriles amarillos y los carriles blancos. Una imagen en escala de grises no proporciona suficiente información para distinguir entre líneas de carril blancas y amarillas.

La computadora interpreta cualquier imagen en escala de grises como una matriz con una entrada para cada píxel de la imagen. Cada imagen tiene 28 x 28 píxeles de alto y ancho, para dar una suma de 784 píxeles. Cada píxel tiene un valor de píxel único asociado. Este valor indica la claridad u oscuridad de ese píxel en particular. Este valor de píxel es un número entero que va de 0 a 255, donde un valor de cero significa más oscuro y 255 es el más blanco, y un píxel gris está entre 0 y 255.



**The simplest artificial neural network**

La siguiente imagen representa una red neuronal simple de dos capas:

![](red_neuronal_simple.png)

La primera capa es la capa de entrada y la última capa es la capa de salida. La capa intermedia es la capa oculta. Si hay más de una capa oculta, dicha red es una red neuronal profunda.

La entrada y salida de cada neurona en la capa oculta está conectada a cada neurona en la siguiente capa. Puede haber cualquier número de neuronas en cada capa dependiendo del problema. Consideremos un ejemplo. El ejemplo simple que quizás ya conozca es el popular reconocimiento de dígitos escritos a mano que detecta un número, digamos 5. Esta red aceptará una imagen de 5 y generará 1 o 0. Un 1 es para indicar que la imagen de hecho es un 5 y 0 en caso contrario. Una vez que se crea la red, hay que entrenarla. Podemos inicializar con pesos aleatorios y luego alimentar muestras de entrada conocidas como el conjunto de datos de entrenamiento. Para cada muestra de entrada, verificamos la salida, calculamos la tasa de error y luego ajustamos los pesos para que cada vez que vea 5, dé como resultado 1 y para todo lo demás, dé un cero. Este tipo de entrenamiento se llama aprendizaje supervisado y el método para ajustar los pesos se llama retropropagación. Al construir modelos de redes neuronales artificiales, una de las consideraciones principales es cómo elegir funciones de activación para capas ocultas y de salida. Las tres funciones de activación más utilizadas son la función sigmoidea, la función tangente hiperbólica y la unidad lineal rectificada (ReLU). La belleza de la función sigmoidea es que su derivada se evalúa en zy simplemente z se multiplica por 1 menos z.

Esto nos ayuda a calcular de manera eficiente los gradientes utilizados en las redes neuronales de una manera conveniente. Si las activaciones de feed-forward de la función logística para una capa dada se mantienen en la memoria, los gradientes para esa capa en particular se pueden evaluar con la ayuda de una simple multiplicación y resta en lugar de implementar y reevaluar la función sigmoidea, ya que requiere exponenciación extra. La siguiente imagen nos muestra la función de activación de ReLU, que es cero cuando x <0 y luego lineal con pendiente 1 cuando x> 0

El ReLU es una función no lineal que calcula la función f (x) = max (0, x). Eso significa que una función ReLU es 0 para entradas negativas yx para todas las entradas x> 0. Esto significa que la activación tiene un umbral de cero (consulte la imagen anterior a la izquierda). TensorFlow implementa la función ReLU en tf.nn.relu ():

![](relu_function.png)

La propagación hacia atrás, una abreviatura de "propagación hacia atrás de errores", es un método común de entrenamiento de redes neuronales artificiales que se utiliza junto con un método de optimización como el descenso de gradientes. El método calcula el gradiente de una función de pérdida con respecto a todos los pesos de la red. El método de optimización se alimenta con el gradiente y lo usa para actualizar los pesos para reducir la función de pérdida.



**Building a single-layer neural network with TensorFlow**


Construyamos una red neuronal de una sola capa con TensorFlow paso a paso. En este ejemplo, usaremos el conjunto de datos MNIST. Este conjunto de datos es un conjunto de imágenes en escala de grises de 28 x 28 píxeles de dígitos escritos a mano. Este conjunto de datos consta de 55,000 datos de entrenamiento, 10,000 datos de prueba y 5,000 datos de validación. Cada punto de datos MNIST tiene dos partes: una imagen de un dígito escrito a mano y una etiqueta correspondiente. El siguiente bloque de código carga datos. one_hot = True significa que las etiquetas son vectores codificados one-hot en lugar de dígitos reales de la etiqueta. Por ejemplo, si la etiqueta es 2, verá [0,0,1,0,0,0,0,0,0,0]. Esto nos permite usarlo directamente en la capa de salida de la red:


In [ ]:
## Version anterior donde se cargan datos y se convierte en formato one hot
##from tensorflow.examples.tutorials.mnist import input_data
##mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
## Carga de datos
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [8]:
len(x_train)

60000

In [ ]:
# All the pixels in the image (28 * 28 = 784)
features_count = 784
# there are 10 digits i.e labels
labels_count = 10 # Cantidad de etiqueta de los datos
batch_size = 128  # Tamaño del lote.
epochs = 10       # Cantidad de epocas con las que se entrenara al modelo
learning_rate = 0.5
features = tf.placeholder(tf.float32, [None,features_count])
labels = tf.placeholder(tf.float32, [None, labels_count])
#Set the weights and biases tensors
weights = tf.Variable(tf.truncated_normal((features_count, labels_count)))
biases = tf.Variable(tf.zeros(labels_count),name='biases')

In [ ]:
# Linear Function WX + b
logits = tf.add(tf.matmul(features, weights),biases)
prediction = tf.nn.softmax(logits)
# Cross entropy
cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
# Training loss
loss = tf.reduce_mean(cross_entropy)
# Initializing all variables
init = tf.global_variables_initializer()
# Determining if the predictions are accurate
is_correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(labels, 1))
# Calculating prediction accuracy
accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

In [ ]:
loss,
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [9]:
#Beginning the session
with tf.Session() as sess:
    # initializing all the variables
    sess.run(init)
    total_batch = int(len(x_train) / batch_size) # len (x_train)
    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            _, c = sess.run([optimizer,loss], feed_dict={features: batch_x,
                                                         labels: batch_y})
            avg_cost += c / total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
    print(sess.run(accuracy, feed_dict={features: mnist.test.images, labels:
mnist.test.labels}))

NameError: name 'mnist' is not defined